# GPU resource optimization

We know from early experiments with pre-embedding texts for indexing into OpeSearch that increasing the embedding batch size does not have a net performance benefit.

Now, the idea is to try embedding several small batches in parallel to see if we can get a speed-up that way.

## 1. Run set-up